In [1]:
import torch
import pandas as pd
import numpy as np
import torchvision.transforms as T
from torch import optim, nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from torchmetrics import Accuracy

In [2]:
def train_one_epoch(model, train_loader, loss_fn, optimizer, epoch=None):
    model.train()
    loss_train = AverageMeter()
    acc = Accuracy('multiclass', num_classes=4, ignore_index=0).to(device)
    with tqdm(train_loader, unit='batch') as tepochs:
        for img, mask in tepochs:
            if epoch is not None:
                tepochs.set_description(f'epoch:{epoch}')
            yp = model(img.to(device))
            yp = yp.reshape(yp.size(0), yp.size(-1), -1)
            mask = mask.to(device).view(mask.size(0), -1)
            loss = loss_fn(yp, mask)
            acc.update(yp, mask)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            tepochs.set_postfix(loss=loss_train.avg, accuracy=acc.compute())
            loss_train.update(loss.item())
    return model, loss_train.avg

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
df = pd.read_csv('data.csv')

In [5]:
def decode(coded, imshape):
    shape = imshape[0] * imshape[1]
    mask = np.asarray(coded.split(), dtype=int)
    st = mask[::2]
    le = mask[1::2]
    ends = st + le  
    imas = np.zeros(shape, dtype=int)
    for low, high in zip(st, ends):
        imas[low:high] = 1
    return imas.reshape(imshape)

In [6]:
def get_segmented_with_same_format(df):
    get_vals = list()
    for i in range(0, len(df), 3):
        bakt = df.iloc[i:i+3].dropna().values
        if bakt.shape[0] != 0:
            get_vals.append(bakt)
    return get_vals

In [7]:
data = get_segmented_with_same_format(df)

In [8]:
transforms = T.Compose([
            T.Resize((256, 256))
])


In [9]:
class MySegDataset(Dataset):
    
    def __init__(self, datas: list):
        classes = {'large_bowel': 1, 'stomach': 2, 'small_bowel' : 3}
        self.datas = datas
        tr = T.ToTensor()
        self.y = list()
        self.x = list()
        for sample in self.datas:
            ims = tr(Image.open(sample[0][-1]))
            decoded_image = torch.zeros((ims.shape[1:]), dtype=torch.long)
             # creating image with labels on each pixel shape 1xhxw background is zero
            for i in sample:
                temp_dec = decode(i[6], ims.shape[1:])
                decoded_image[temp_dec == 1] = classes[i[3]]
            self.y.append(transforms(decoded_image.unsqueeze(0))) 
            self.x.append(transforms(ims / 255))  # main Image
    def __getitem__(self, idx):
        return self.x[idx],  self.y[idx]
    
    def __len__(self):
        return len(self.x)
    

In [10]:
data_set = MySegDataset(data)

/home/vahid/anaconda3/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [11]:
train_loader = DataLoader(data_set, batch_size=30, shuffle=True)

In [12]:
def block(in_f, out_f):
      return nn.Sequential(
                            nn.Conv2d(in_f, out_f, kernel_size=3, padding=1),
                            nn.ReLU(),
                            nn.BatchNorm2d(out_f),
                            nn.Conv2d(out_f, out_f, kernel_size=3, padding=1),
                            nn.ReLU(),
                            nn.BatchNorm2d(out_f),
                    )
  

In [13]:
class UBadNet(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        # down sample, decoder phase
        self.block1 = block(1, 64)
        self.block2 = block(64, 128)
        self.block3 = block(128, 256)
        self.block4 = block(256, 512)
        self.block5 = block(512, 1024)
        self.block6 = block(1024, 1024)
        
        # Going N_block for concated tensors on decoder
        self.n_bloc_1 = block(1024, 512)
        self.n_bloc_2 = block(512, 256)
        self.n_bloc_3 = block(256, 128)
        self.n_bloc_4 = block(128, 64)

        # up sample 
        self.upc_1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.upc_2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.upc_3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.upc_4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)

        self.decoder = nn.Conv2d(64, 4, kernel_size=3, padding=1)
        self.mp = nn.MaxPool2d(kernel_size=3, padding=1, stride=2)
        
        
    def forward(self, x):
        x_1 = self.block1(x) # 256 64
        x_m =  self.mp(x_1) # 128 64
        x_2 = self.block2(x_m) # 128 128
        x_m = self.mp(x_2) # 64 128
        x_3 = self.block3(x_m) # 64 256
        x_m = self.mp(x_3) # 32 256
        x_4 = self.block4(x_m) # 32 512
        x_m = self.mp(x_4) # 16 512
        x_5 = self.block5(x_m) # 16 1024
        
        y_temp = self.upc_1(x_5) # 32 512
        y = self.n_bloc_1(torch.cat([y_temp, x_4], dim=1)) # 32, 1024 > 32 512
        y_temp = self.upc_2(y) # 64 256
        y = self.n_bloc_2(torch.cat([y_temp, x_3], dim=1)) # 64, 512 > 64 256
        y_temp = self.upc_3(y) # 128 128
        y = self.n_bloc_3(torch.cat([y_temp, x_2], dim=1)) # 128, 256 >  128, 128
        y_temp = self.upc_4(y) # 256, 64
        y = self.n_bloc_4(torch.cat([y_temp, x_1], dim=1)) # 256, 128 > 256, 64
        y = self.decoder(y) # going to be 256, num_classes
        y = y.permute(0, 2, 3, 1) 
        return y

In [15]:
model = UBadNet().to(device)

In [16]:
sum(p.numel() for p in model.parameters())

49925124

In [17]:
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(ignore_index=0)

In [18]:
train_one_epoch(model, train_loader, loss_fn, optimizer)

  0%|                                                | 0/553 [00:00<?, ?batch/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 480.00 MiB. GPU 0 has a total capacty of 9.74 GiB of which 128.12 MiB is free. Including non-PyTorch memory, this process has 8.99 GiB memory in use. Of the allocated memory 8.35 GiB is allocated by PyTorch, and 392.05 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF